# IML NET

In [1]:
from model import IMLE_net
from model.IMLE_net.inference import build_scores, transform_ecg_for_inference
import os
import numpy as np
from data import ptbxl
from utils import setCWDToProjectDir
from model.IMLE_net.preprocessing.preprocess import preprocess
import tensorflow as tf

In [ ]:
# See if GPU is available
print(tf.config.list_physical_devices('GPU'))

In [2]:
setCWDToProjectDir()
print(f"Our current working directory is {os.getcwd()}")

Our current working directory is D:\cardiovascular-ai


## Author's model and weight

In [ ]:


dataset = ptbxl.PTBXL(sampling_rate=ptbxl.SamplingRate.HZ_100)
record = dataset.load_record(1)
weight_path = os.path.join(os.getcwd(), 'ckpts', 'imle_net_weights.h5')

config = IMLE_net.Config()

# Set the configuration parameters for the Author's model (imle_net_weights)
config.attention_dim = 32
config.lstm_units = 32
config.start_filters = 16

#Print all config values
print("Model configuration:")
for key, value in config.__dict__.items():
    print(f"\t {key}: {value}")
print("")
model_author = IMLE_net.build_imle_net(config)
model_author.load_weights(weight_path)

data = record.data
data = np.expand_dims(data, axis=0)
data = transform_ecg_for_inference(data)
print("Data shape after transformation:", data.shape)
build_scores(model_author, data, config)





## Community model and weight

In [ ]:


dataset = ptbxl.PTBXL(sampling_rate=ptbxl.SamplingRate.HZ_100)
record = dataset.load_record(1)
weight_path = os.path.join(os.getcwd(), 'ckpts', 'imle_net_community_weights.h5')

config = IMLE_net.Config()

#Print all config values
print("Model configuration:")
for key, value in config.__dict__.items():
    print(f"\t {key}: {value}")
print("")
model = IMLE_net.build_imle_net(config)
model.load_weights(weight_path)

data = record.data
data = np.expand_dims(data, axis=0)
data = transform_ecg_for_inference(data)
print("Data shape after transformation:", data.shape)
build_scores(model, data, config)





In [ ]:
predictions = model_author.predict(data)
print("Predictions shape: ", predictions.shape)
# Print predicted class
predicted_class = np.argmax(predictions, axis=1)
print("Prediction: ", predicted_class[0])

#print probabilities
print("Probabilities: ", predictions[0])


In [ ]:
preprocessed_file_path = f"preprocessed_data/ptbxl_{dataset.sampling_rate}.npz"

if not os.path.exists(preprocessed_file_path):
  X, y, data, classes = dataset.load_all()
  
  #Print classes
  print("Classes: ", classes)

  #preprocess
  X_train_scale, y_train, X_test_scale, y_test, X_val_scale, y_val = preprocess(X, y, data)

  print("X_train_scale shape: ", X_train_scale.shape)
  print("y_train shape: ", y_train.shape)
  print("X_test_scale shape: ", X_test_scale.shape)
  print("y_test shape: ", y_test.shape)
  print("X_val_scale shape: ", X_val_scale.shape)
  print("y_val shape: ", y_val.shape)

  # Save the preprocessed data
  
  # Create the directory if it doesn't exist
  os.makedirs(os.path.dirname(preprocessed_file_path), exist_ok=True)

  # Save the preprocessed data to a compressed .npz file
  np.savez_compressed(preprocessed_file_path, X_train_scale=X_train_scale, y_train=y_train, \
              X_test_scale=X_test_scale, y_test=y_test, X_val_scale=X_val_scale, y_val=y_val,\
              classes=classes)
else:
  print(f"Preprocessed data already exists at {preprocessed_file_path}. Loading it...")

# Load the preprocessed data
data = np.load(preprocessed_file_path, allow_pickle=True)
X_train_scale = data['X_train_scale']
y_train = data['y_train']
X_test_scale = data['X_test_scale']
y_test = data['y_test']
X_val_scale = data['X_val_scale']
y_val = data['y_val']

classes = data['classes']
print("Loaded classes: ", classes)

In [ ]:
X_test = transform_ecg_for_inference(X_test_scale, scaler_path=None)
community_predictions = model.predict(X_test)
author_predictions = model_author.predict(X_test)

In [ ]:
from utils.display import display_confusion_matrix
from utils.evaluation import evaluate_classification

print("Shape of community predictions: ", community_predictions.shape)

evaluate_classification(community_predictions, X_test, y_test, classes)

In [ ]:
evaluate_classification(author_predictions, X_test, y_test)

In [ ]:
dataset = ptbxl.PTBXL(sampling_rate=ptbxl.SamplingRate.HZ_100)
X, y, data, classes = dataset.load_all()
X_test = transform_ecg_for_inference(X)
# author_predictions = model_author.predict(X_test)
# community_predictions = model.predict(X_test)
# print("Shape of community predictions: ", community_predictions.shape)
# evaluate_classification(community_predictions, X_test, y, classes=classes)
# evaluate_classification(author_predictions, X_test, y, classes=classes)

Loading dataset using multiple threads...



AttributeError: Can't pickle local object 'PTBXL._load_raw_data_mp.<locals>.read_file'